Read in a single file and make duplicates

In [2]:
import pyspark
from delta import *
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql.window import Window
import json
spark = (pyspark.sql.SparkSession.builder.appName("MyApp").master("local[*]")
    .config("spark.jars.packages", "io.delta:delta-spark_2.12:3.1.0")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.S3SingleDriverLogStore")
.getOrCreate())
spark.sparkContext.setLogLevel('ERROR')

:: loading settings :: url = jar:file:/home/spark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sparkuser/.ivy2/cache
The jars for the packages stored in: /home/sparkuser/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-c6388755-197b-4974-90cd-ff7fa07bfa6d;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.1.0 in central
	found io.delta#delta-storage;3.1.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
downloading https://repo1.maven.org/maven2/io/delta/delta-spark_2.12/3.1.0/delta-spark_2.12-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-spark_2.12;3.1.0!delta-spark_2.12.jar (1900ms)
downloading https://repo1.maven.org/maven2/io/delta/delta-storage/3.1.0/delta-storage-3.1.0.jar ...
	[SUCCESSFUL ] io.delta#delta-storage;3.1.0!delta-storage.jar (82ms)
downloading https://repo1.maven.org/maven2/org/antlr/antlr4-runtime/4.9.3/antlr4-runtime-4.9.3.jar ...
	[SUCCESSFUL ] org.antlr#antlr4-runtime;4.9.3!antlr4-runtime.jar (96ms)
:: resolution report :: resolve 2175

In [3]:
df_array = ["""{"id": "2111", "name": "OLIVIA", "age": "37", "metadata":[["a",1], ["b",2]]}"""]
df_array = spark.sparkContext.parallelize(df_array)
df_array = spark.read.json(df_array)
df_array.show()

+---+----+----------------+------+
|age|  id|        metadata|  name|
+---+----+----------------+------+
| 37|2111|[[a, 1], [b, 2]]|OLIVIA|
+---+----+----------------+------+



In [5]:
spark.sql("DROP TABLE IF EXISTS spark")

DataFrame[]

In [6]:
df_array.write.format("delta").mode("overwrite").save("spark")

In [7]:
df_array_2 = df_array.withColumn("name", F.lower(F.col("name")))

In [8]:
df_array.write.format("delta").mode("overwrite").save("spark")

In [9]:
spark.read.format("delta").load("spark").show()

+---+----+----------------+------+
|age|  id|        metadata|  name|
+---+----+----------------+------+
| 37|2111|[[a, 1], [b, 2]]|OLIVIA|
+---+----+----------------+------+



know how files are layed out in the delta lakes format

In [10]:
!ls spark

_delta_log
part-00000-16b0701f-fa27-438f-9542-c86a5c2133d6-c000.snappy.parquet
part-00000-1f324c2a-50c7-4ccc-9ef2-72440ec5997a-c000.snappy.parquet
part-00000-240124c6-6773-4756-816d-5b8f6fe23921-c000.snappy.parquet
part-00000-3fc3639d-e0c0-4c97-ba3c-da1317dd3f16-c000.snappy.parquet
part-00000-49ac9f2d-6366-4d82-83d4-11eaa49e8b3c-c000.snappy.parquet
part-00000-57c8f533-8d58-4f95-a573-6f2528ad667e-c000.snappy.parquet
part-00000-5d7596ad-c915-478a-bc7e-6f97beb05d7e-c000.snappy.parquet
part-00000-9c446b7c-32bb-405e-8fc4-ce272125d484-c000.snappy.parquet
part-00000-b1396d66-41bd-4bb0-8ccc-70dce053ef8a-c000.snappy.parquet
part-00000-e40fe0eb-2626-4b54-adb8-8a619136b72b-c000.snappy.parquet
part-00000-f1885439-8175-4f07-8558-44cb87f666a3-c000.snappy.parquet
part-00007-0e3f3116-ec64-4ccd-8dfa-b119b5387d4b-c000.snappy.parquet
part-00007-1762504f-5eb6-446c-889a-9ec6814b8d05-c000.snappy.parquet
part-00007-700f149a-3f26-44d2-9acb-99e96abfce03-c000.snappy.parquet
part-00007-72364959-5a66-4543-8da3-19

In [11]:
!ls spark/_delta_log

00000000000000000000.json  00000000000000000007.json
00000000000000000001.json  00000000000000000008.json
00000000000000000002.json  00000000000000000009.json
00000000000000000003.json  00000000000000000010.checkpoint.parquet
00000000000000000004.json  00000000000000000010.json
00000000000000000005.json  _last_checkpoint
00000000000000000006.json


look at the tables history

In [13]:
from IPython.display import display, HTML
from delta.tables import DeltaTable
import pandas as pd

delta_table = DeltaTable.forPath(spark, "spark")

history_df = delta_table.history()

history_df.toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,10,2025-02-13 17:51:41.670,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,9.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
1,9,2025-02-13 17:51:40.706,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,8.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
2,8,2025-02-13 17:16:44.839,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,7.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
3,7,2025-02-13 17:16:43.990,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,6.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
4,6,2025-02-13 17:01:04.787,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,5.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
5,5,2025-02-13 17:01:03.937,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,4.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
6,4,2025-02-13 17:00:34.811,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,3.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
7,3,2025-02-13 17:00:33.899,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,2.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
8,2,2025-02-13 16:59:30.985,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
9,1,2025-02-13 05:15:20.309,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,Serializable,False,"{'numOutputRows': '1', 'numOutputBytes': '2076...",None,Apache-Spark/3.5.4 Delta-Lake/3.1.0
